# Práctica inicial 
## TP integrador - Primera parte
### EDA inicial - Obras Públicas (Dataset Provincia Bs. As)
#### Grupo_08 - Septiembre 2020

In [ ]:
# importamos las librerías necesarias para trabajar.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Una vez que descargamos el dataset procedemos a leerlo
obras = pd.read_csv('C:/Users/Mica/Google Drive/Quinto año/Cs de datos/Práctica/observatorio-de-obras-urbanas.csv')

In [ ]:
print(f'Shape: {np.shape(obras)}')
obras.head()

In [ ]:
columnas = obras.columns
columnas

In [ ]:
col_nulls = obras.isnull().any()
col_nulls

In [ ]:
tipos_obras = len(obras['tipo'].unique()) # Cantidad de valores unicos 
print(f'En total hay {tipos_obras} tipos de obras."\r\n')
print("\n", obras['tipo'].unique())

In [ ]:
etapas= len(obras['etapa'].unique()) # Cantidad de valores unicos 
print(f'En total hay {etapas} etapas de obras."\r\n')
print("\n", obras['etapa'].unique())

In [ ]:
fecha= len(obras['fecha_inicio'].unique()) # Cantidad de valores unicos 
print(f'En total hay {fecha} etapas de obras."\r\n')
print("\n", obras['fecha_inicio'].unique())

In [ ]:
plazo_meses=len(obras['plazo_meses'].unique()) # Cantidad de valores unicos 
print(f'En total hay {plazo_meses} etapas de obras."\r\n')
print("\n", obras['plazo_meses'].unique())

In [ ]:
barrio=len(obras['barrio'].unique()) # Cantidad de valores unicos 
print(f'En total hay {barrio} barrios."\r\n')
print("\n", obras['barrio'].unique())

In [ ]:
ax = sns.countplot(x='tipo', data=obras)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.title("Hola")
plt.show()

In [ ]:
ax = sns.countplot(x='etapa', data=obras)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.title("Hola")
plt.show()

In [ ]:
ax = sns.countplot(x='comuna', data=obras)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
plt.title("Hola")
plt.show()